In [1]:
!pip install transformers
!pip install tensorboardx

In [4]:
!pip install chardet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 4.7 MB/s eta 0:00:0000:01


In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.backend_bases import RendererBase
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils import data
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from tensorboardX import SummaryWriter
from transformers import BertModel, BertTokenizer
import torch
import csv

# File util

In [5]:
#-*- coding: utf-8 -*-


import os
import chardet

'''
dirname        : path that need to be searched
ret                : files in the dirname (recursive)
list_avoid_dir : dirname need to be skipped
usage           : 
    list_files = []
    file_search(dirname, list_files):   
'''
def file_search(dirname, ret, list_avoid_dir=[]):
    
    filenames = os.listdir(dirname)
    
    for filename in filenames:
        full_filename = os.path.join(dirname, filename)

        if os.path.isdir(full_filename) :
            if full_filename.split('/')[-1] in list_avoid_dir:
                continue
            else:
                file_search(full_filename, ret, list_avoid_dir)
            
        else:
            ret.append( full_filename )          

            

'''
filename : filename (inc. path) that will be inspected
'''
def find_encoding(filename):
    rawdata = open(filename, 'rb').read()
    result = chardet.detect(rawdata)
    charenc = result['encoding']    
    return charenc
            
'''
dir_name : dir_name (inc. path) that will be created ( full-path name )
'''
def create_folder(dir_name):
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)

# Extract Label Transcription

In [8]:
out_file = '/kaggle/working/processed_tran.csv'
os.system('rm ' + out_file)
def extract_trans( list_in_file, out_file ) :
    lines = []
    for in_file in list_in_file:
        cnt = 0
        
        with open(in_file, 'r') as f:
            lines = f.readlines()

        with open(out_file, 'a') as f:

            csv_writer = csv.writer( f )
            lines = sorted(lines)                  # sort based on first element
            
            for line in lines:

                name = line.split(':')[0].split(' ')[0].strip()
                
                # unwanted case 
                if name[:3] != 'Ses':             # noise transcription such as reply  M: sorry
                    continue
                elif name[-3:-1] == 'XX':        # we don't have matching pair in label
                    continue
                trans = line.split(':')[1].strip()
                
                cnt += 1
                csv_writer.writerow([name, trans])

list_files = []

for x in range(5):
    sess_name = 'Session' + str(x+1)

    path = '/kaggle/input/iemocapfullrelease/IEMOCAP_full_release/' + sess_name + '/dialog/transcriptions/'
    file_search(path, list_files)
    list_files = sorted(list_files)

    print(sess_name + ", #sum files: " + str(len(list_files)))

extract_trans(list_files, out_file)

Session1, #sum files: 28
Session2, #sum files: 58
Session3, #sum files: 90
Session4, #sum files: 120
Session5, #sum files: 151


In [20]:
df=pd.read_csv('/kaggle/working/processed_tran.csv')

In [21]:
df.head()

,Ses01F_impro01_F000,Excuse me.
0,Ses01F_impro01_F001,Yeah.
1,Ses01F_impro01_F002,Is there a problem?
2,Ses01F_impro01_F003,You did.
3,Ses01F_impro01_F004,You were standing at the beginning and you dir...
4,Ses01F_impro01_F005,Well what's the problem? Let me change it.


In [22]:
out_file = '/kaggle/working/label.csv'
list_category = [
                'ang',
                'hap',
                'sad',
                'neu',
                'fru',
                'exc',
                'fea',
                'sur',
                'dis',
                'oth',
                'xxx'
                ]

category = {}
for c_type in list_category:
    if c_type in category:
        ;
    else:
        category[c_type] = len(category)

In [23]:
def find_category(lines):
    is_target = True
    
    id = ''
    c_label = ''
    list_ret = []
    
    for line in lines:
        
        if is_target == True:
            
            try:
                id = line.split('\t')[1].strip()  #  extract ID
                c_label  = line.split('\t')[2].strip()  #  extract category
                if c_label not in category:
                    print("ERROR nokey" + c_label)
                    sys.exit()
                
                list_ret.append( [id, c_label] )
                is_target = False

            except:
                print("ERROR " + line)
                sys.exit()
        
        else:
            if line == '\n':
                is_target = True
    return list_ret

def extract_labels( list_in_file, out_file ) :
    id = ''
    lines = []
    list_ret = []
    
    for in_file in list_in_file:
        
        with open(in_file, 'r') as f:
            lines = f.readlines()
            lines = lines[2:]                           # remove head
            list_ret = find_category(lines)
            
        list_ret = sorted(list_ret)                   # sort based on first element
    
        with open(out_file, 'a') as f:
            csv_writer = csv.writer( f )
            csv_writer.writerows( list_ret )  

In [24]:
list_files = []
list_avoid_dir = ['Attribute', 'Categorical', 'Self-evaluation']

for x in range(5):
    sess_name = 'Session' + str(x+1)

    path = '/kaggle/input/iemocapfullrelease/IEMOCAP_full_release/' + sess_name + '/dialog/EmoEvaluation/'
    file_search(path, list_files, list_avoid_dir)
    list_files = sorted(list_files)

    print(sess_name + ", #sum files: " + str(len(list_files)))

extract_labels(list_files, out_file)

Session1, #sum files: 28
Session2, #sum files: 58
Session3, #sum files: 90
Session4, #sum files: 120
Session5, #sum files: 151


In [25]:
lines = []
with open('/kaggle/working/label.csv') as f :
    csv_reader = csv.reader(f)
    lines = [x for x in csv_reader]
df = pd.DataFrame(columns=['sessionID', 'label'])
with open('/kaggle/working/processed_label.txt', 'w') as f:
    
    with open('/kaggle/working/processed_ids.txt', 'w') as f2:
    
        for line in lines:
            if line[1] == 'ang':
                f.write('0\n')
                f2.write(line[0]+'\n')
                df.loc[len(df.index)] = [line[0], 0] 
            elif line[1] == 'hap':
                f.write('1\n')
                f2.write(line[0]+'\n')
                df.loc[len(df.index)] = [line[0], 1] 
            elif line[1] == 'exc':
                f.write('1\n')
                f2.write(line[0]+'\n')
                df.loc[len(df.index)] = [line[0], 1] 
            elif line[1] == 'sad':
                f.write('2\n')
                f2.write(line[0]+'\n')
                df.loc[len(df.index)] = [line[0], 2] 
            elif line[1] == 'neu':
                f.write('3\n')
                f2.write(line[0]+'\n')
                df.loc[len(df.index)] = [line[0], 3] 
            else :
                f.write('-1\n')
                df.loc[len(df.index)] = [line[0], -1] 

In [26]:
lines = []
with open('/kaggle/working/processed_label.txt') as f :
    lines = f.readlines()
lines = [x.strip() for x in lines]

print(len([x for x in lines if x=='0']))
print(len([x for x in lines if x=='1']))
print(len([x for x in lines if x=='2']))
print(len([x for x in lines if x=='3']))

1103
1636
1084
1708


'3'

In [43]:
df=pd.read_csv('/kaggle/working/processed_tran.csv')
temp = pd.DataFrame(data=df.values, columns=['sessionID', 'text'])
new_row = pd.DataFrame({'sessionID':'Ses01F_impro01_F000', 'text': 'Excuse me.'}, index=[0])
df = pd.concat([new_row,temp.loc[:]]).reset_index(drop=True)
df["label"] = lines
df.head()

,sessionID,text,label
0,Ses01F_impro01_F000,Excuse me.,3
1,Ses01F_impro01_F001,Yeah.,3
2,Ses01F_impro01_F002,Is there a problem?,3
3,Ses01F_impro01_F003,You did.,-1
4,Ses01F_impro01_F004,You were standing at the beginning and you dir...,-1


In [48]:
df.shape

(10039, 3)

In [57]:
df['label'] = pd.to_numeric(df['label'])

In [58]:
docs=[]
for text,label in zip(df['text'],df['label']):
  if(label!=-1):
    docs.append({'text':text,'label':label})
print(len(docs))

5531


# Creating train and test set

In [59]:
import random
random.shuffle(docs)
random.shuffle(docs)
random.shuffle(docs)
total_length=len(docs)
train_length=int(.9*total_length)
train_list=docs[0:train_length]
test_list=docs[train_length:]
print('no of items for train ',len(train_list))
print('no of items for test ',len(test_list))

no of items for train  4977
no of items for test  554


# BERT model for classification

In [60]:
def weight_init(m):
    if isinstance(m, torch.nn.Linear):
        print('init of linear is done')
        torch.nn.init.xavier_uniform_(m.weight)
        if m.bias is not None: 
            torch.nn.init.xavier_uniform_(m.bias)

In [52]:
%load_ext tensorboard
%tensorboard --logdir ./

In [66]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels = 4,   
    output_attentions = False,
    output_hidden_states = False, 
)
print(model)
params = list(model.named_parameters())
optimizer = AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8 
                )
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
NUM_EPOCHS=4


writer = SummaryWriter(log_dir='/kaggle/working/content/')
total_steps = len(train_list) * NUM_EPOCHS

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [67]:
import random
total_steps = 1
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

model.to('cuda')
for epoch in range(NUM_EPOCHS):
  model.train()
  random.shuffle(train_list)
  for every_trainlist in train_list:
    label1=every_trainlist['label']
    label1=torch.tensor([label1])
    text=every_trainlist['text']
    input_ids = torch.tensor(tokenizer.encode(text, add_special_tokens=True)).unsqueeze(0)  # Batch size 1
    
    model.zero_grad()
    input_ids = input_ids.to('cuda')
    label1=label1.to('cuda')
    
    outputs = model(input_ids, 
            token_type_ids=None,
            labels=label1)
    loss = outputs[0]
    logits = outputs[1]

#     total_loss += loss.item()

#     loss.backward()

#     torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

#     optimizer.step()

#     scheduler.step()
    
#     loss,logits = model(input_ids,labels=label1)
    print('loss.item()',loss.item())
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    scheduler.step()
    
    _, preds = torch.max(logits, 1)
    print('preds',preds)
    accuracy = torch.sum(preds == label1)
    print('accuracy.item()',accuracy.item())
    
    if total_steps % 10 == 0:
      with torch.no_grad():
        _, preds = torch.max(logits, 1)
        accuracy = torch.sum(preds == label1)
        writer.add_scalar('training loss', loss.item(), total_steps)
        writer.add_scalar('training accuracy', accuracy.item(), total_steps)                     
    total_steps+=1

loss.item() 1.6661100387573242
preds tensor([3], device='cuda:0')
accuracy.item() 0
loss.item() 0.7920992970466614
preds tensor([3], device='cuda:0')
accuracy.item() 1
loss.item() 1.5648363828659058
preds tensor([3], device='cuda:0')
accuracy.item() 0
loss.item() 0.7791898250579834
preds tensor([3], device='cuda:0')
accuracy.item() 1
loss.item() 1.5936896800994873
preds tensor([3], device='cuda:0')
accuracy.item() 0
loss.item() 1.7950444221496582
preds tensor([3], device='cuda:0')
accuracy.item() 0
loss.item() 1.6743866205215454
preds tensor([3], device='cuda:0')
accuracy.item() 0
loss.item() 1.4516825675964355
preds tensor([3], device='cuda:0')
accuracy.item() 0
loss.item() 2.022876262664795
preds tensor([3], device='cuda:0')
accuracy.item() 0
loss.item() 1.3794217109680176
preds tensor([3], device='cuda:0')
accuracy.item() 0
loss.item() 2.0689754486083984
preds tensor([3], device='cuda:0')
accuracy.item() 0
loss.item() 1.8275930881500244
preds tensor([3], device='cuda:0')
accuracy.it

In [68]:
y_actu=[]
y_pred=[]
model.to('cpu')
model.eval()
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
for every_test_list in test_list:
    label1=every_test_list['label']
    label1=torch.tensor([label1])
    text=every_test_list['text']
    input_ids = torch.tensor(tokenizer.encode(text, add_special_tokens=True)).unsqueeze(0)  # Batch size 1
    with torch.no_grad():
        outputs = model(input_ids, 
            token_type_ids=None,
            labels=label1)
        loss = outputs[0]
        logits = outputs[1]
#       loss, output = model(input_ids,labels=label1)
        _, preds = torch.max(logits, 1)
        y_actu.append(label1.numpy()[0])
        y_pred.append(preds.numpy()[0])

In [69]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_actu, y_pred)

array([[ 83,   3,   7,  13],
       [  7, 128,  10,  30],
       [  4,   7,  73,  12],
       [ 18,  33,  15, 111]])

In [71]:
from sklearn.metrics import classification_report
print(classification_report(y_actu,y_pred))

              precision    recall  f1-score   support

           0       0.74      0.78      0.76       106
           1       0.75      0.73      0.74       175
           2       0.70      0.76      0.73        96
           3       0.67      0.63      0.65       177

    accuracy                           0.71       554
   macro avg       0.71      0.73      0.72       554
weighted avg       0.71      0.71      0.71       554



In [70]:
torch.save(model, '/kaggle/working//model_text.pt')